This code takes the reprojected carbon stock layers and calculates zonal statistics across the y2y region based on ecoregion and biome attributes.

In [17]:
# import packages
import geopandas as gpd
import numpy as np
import rioxarray as rxr
import xarray

import pandas as pd
from pandas.api.types import is_numeric_dtype

from geocube.api.core import make_geocube

import warnings

In [18]:
# load 2017 ecoregions clipped to y2y and reproject to match rasters
eco = gpd.read_file('./land_cover/ecoregions2017_clipped.shp').to_crs(
    '+proj=laea +lat_0=55 +lon_0=-125 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs')
eco

,OBJECTID,ECO_NAME,BIOME_NUM,BIOME_NAME,REALM,ECO_BIOME_,NNH,ECO_ID,SHAPE_LENG,SHAPE_AREA,NNH_NAME,COLOR,COLOR_BIO,COLOR_NNH,LICENSE,geometry
0,13.0,Alberta-British Columbia foothills forests,5.0,Temperate Conifer Forests,Nearctic,NE05,2,345,62.333821,17.133639,Nature Could Reach Half Protected,#5DAD4C,#458970,#7BC141,CC-BY 4.0,"MULTIPOLYGON (((231107.227 305009.454, 226734...."
1,74.0,Blue Mountains forests,5.0,Temperate Conifer Forests,Nearctic,NE05,3,348,22.624165,8.066966,Nature Could Recover,#35A85A,#458970,#F9A91B,CC-BY 4.0,"POLYGON ((614252.395 -1139094.041, 612099.656 ..."
2,83.0,British Columbia coastal conifer forests,5.0,Temperate Conifer Forests,Nearctic,NE05,2,349,86.917940,14.653986,Nature Could Reach Half Protected,#00A884,#458970,#7BC141,CC-BY 4.0,"POLYGON ((-166421.245 37682.88, -177859.583 32..."
3,84.0,Brooks-British Range tundra,11.0,Tundra,Nearctic,NE11,1,411,87.112754,34.783625,Half Protected,#66BECA,#9ED7C2,#257339,CC-BY 4.0,"MULTIPOLYGON (((-514179.871 1392834.151, -5147..."
4,95.0,Canadian Aspen forests and parklands,8.0,"Temperate Grasslands, Savannas & Shrublands",Nearctic,NE08,4,386,86.836978,40.677412,Nature Imperiled,#DB931A,#FEFF73,#EE1E23,CC-BY 4.0,"MULTIPOLYGON (((725180.479 -339910.34, 723716...."
5,133.0,Central British Columbia Mountain forests,5.0,Temperate Conifer Forests,Nearctic,NE05,2,350,36.323846,19.868732,Nature Could Reach Half Protected,#4CD970,#458970,#7BC141,CC-BY 4.0,"POLYGON ((192654.361 -120678.524, 194329.256 -..."
6,272.0,Fraser Plateau and Basin conifer forests,5.0,Temperate Conifer Forests,Nearctic,NE05,3,355,26.796172,13.925479,Nature Could Recover,#5ABD7A,#458970,#F9A91B,CC-BY 4.0,"POLYGON ((298504.267 -341027.666, 298711.41 -3..."
7,337.0,Interior Alaska-Yukon lowland taiga,6.0,Boreal Forests/Taiga,Nearctic,NE06,2,375,199.626313,80.342543,Nature Could Reach Half Protected,#6D3EC2,#7AB6F5,#7BC141,CC-BY 4.0,"MULTIPOLYGON (((-627905.705 1421020.585, -6225..."
8,339.0,Interior Yukon-Alaska alpine tundra,11.0,Tundra,Nearctic,NE11,2,416,121.006872,29.065154,Nature Could Reach Half Protected,#6EE3C6,#9ED7C2,#7BC141,CC-BY 4.0,"MULTIPOLYGON (((-713662.418 1111909.509, -7138..."
9,438.0,Mid-Canada Boreal Plains forests,6.0,Boreal Forests/Taiga,Nearctic,NE06,2,376,109.724632,79.516771,Nature Could Reach Half Protected,#DCB1F9,#7AB6F5,#7BC141,CC-BY 4.0,"MULTIPOLYGON (((253378.224 142341.081, 266856...."


In [19]:
# filter by columns needed
eco = eco.filter(
    ['OBJECTID', 'ECO_NAME', 'BIOME_NUM', 'BIOME_NAME', 'geometry'])
eco

,OBJECTID,ECO_NAME,BIOME_NUM,BIOME_NAME,geometry
0,13.0,Alberta-British Columbia foothills forests,5.0,Temperate Conifer Forests,"MULTIPOLYGON (((231107.227 305009.454, 226734...."
1,74.0,Blue Mountains forests,5.0,Temperate Conifer Forests,"POLYGON ((614252.395 -1139094.041, 612099.656 ..."
2,83.0,British Columbia coastal conifer forests,5.0,Temperate Conifer Forests,"POLYGON ((-166421.245 37682.88, -177859.583 32..."
3,84.0,Brooks-British Range tundra,11.0,Tundra,"MULTIPOLYGON (((-514179.871 1392834.151, -5147..."
4,95.0,Canadian Aspen forests and parklands,8.0,"Temperate Grasslands, Savannas & Shrublands","MULTIPOLYGON (((725180.479 -339910.34, 723716...."
5,133.0,Central British Columbia Mountain forests,5.0,Temperate Conifer Forests,"POLYGON ((192654.361 -120678.524, 194329.256 -..."
6,272.0,Fraser Plateau and Basin conifer forests,5.0,Temperate Conifer Forests,"POLYGON ((298504.267 -341027.666, 298711.41 -3..."
7,337.0,Interior Alaska-Yukon lowland taiga,6.0,Boreal Forests/Taiga,"MULTIPOLYGON (((-627905.705 1421020.585, -6225..."
8,339.0,Interior Yukon-Alaska alpine tundra,11.0,Tundra,"MULTIPOLYGON (((-713662.418 1111909.509, -7138..."
9,438.0,Mid-Canada Boreal Plains forests,6.0,Boreal Forests/Taiga,"MULTIPOLYGON (((253378.224 142341.081, 266856...."


In [20]:
# rename id columns
eco = eco.rename(columns={'OBJECTID': 'ECO_ID', 'BIOME_NUM': 'BIOME_ID'})
eco

,ECO_ID,ECO_NAME,BIOME_ID,BIOME_NAME,geometry
0,13.0,Alberta-British Columbia foothills forests,5.0,Temperate Conifer Forests,"MULTIPOLYGON (((231107.227 305009.454, 226734...."
1,74.0,Blue Mountains forests,5.0,Temperate Conifer Forests,"POLYGON ((614252.395 -1139094.041, 612099.656 ..."
2,83.0,British Columbia coastal conifer forests,5.0,Temperate Conifer Forests,"POLYGON ((-166421.245 37682.88, -177859.583 32..."
3,84.0,Brooks-British Range tundra,11.0,Tundra,"MULTIPOLYGON (((-514179.871 1392834.151, -5147..."
4,95.0,Canadian Aspen forests and parklands,8.0,"Temperate Grasslands, Savannas & Shrublands","MULTIPOLYGON (((725180.479 -339910.34, 723716...."
5,133.0,Central British Columbia Mountain forests,5.0,Temperate Conifer Forests,"POLYGON ((192654.361 -120678.524, 194329.256 -..."
6,272.0,Fraser Plateau and Basin conifer forests,5.0,Temperate Conifer Forests,"POLYGON ((298504.267 -341027.666, 298711.41 -3..."
7,337.0,Interior Alaska-Yukon lowland taiga,6.0,Boreal Forests/Taiga,"MULTIPOLYGON (((-627905.705 1421020.585, -6225..."
8,339.0,Interior Yukon-Alaska alpine tundra,11.0,Tundra,"MULTIPOLYGON (((-713662.418 1111909.509, -7138..."
9,438.0,Mid-Canada Boreal Plains forests,6.0,Boreal Forests/Taiga,"MULTIPOLYGON (((253378.224 142341.081, 266856...."


In [21]:
# define function to extract zonal stats
def extract_stats(dat_fp, dat_name, vector, vect_var, stat):

    # Suppress UserWarning within this function
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)

        # create new vect variable so don't edit data in place
        vect = vector.copy()

        # check if vect_var column is numeric
        numeric = True
        if not is_numeric_dtype(vect[vect_var]):
            vect['key'] = pd.factorize(vect[vect_var])[0]
            vect.rename(columns={vect_var: 'orig_name',
                        'key': vect_var}, inplace=True)
            numeric = False

        # clip data to vector layer
        dat = rxr.open_rasterio(dat_fp, masked=True
                                ).rio.clip(vect.geometry.values, vect.crs, from_disk=True)
        dat.name = dat_name

        # create output grid
        out_grid = make_geocube(
            vector_data=vect,
            measurements=[vect_var],
            like=dat
        )

        # merge the datacube with the data
        out_grid[dat_name] = (dat.dims, dat.values,
                              dat.attrs, dat.encoding)

        # group data by vector variable
        grouped = out_grid.drop_vars(['spatial_ref']).groupby(vect_var)

        # calculate stats
        if stat == 'sum':
            table = grouped.sum()
        if stat == 'mean':
            table = grouped.mean()

        # reset indicies and drop band
        table = table.to_dataframe()
        table.reset_index(level='band', drop=True, inplace=True)

        # reset keys to original values
        if numeric == False:
            mapping = dict(zip(vect[vect_var], vect['orig_name']))
            table.index = table.index.map(mapping)

        # return table
        return table

In [22]:
# load carbon rasters in format for zonal stats function
# file paths
carbon_fp = ['./carbon_stock_data/output_layers/carbon_sothe_spawn_t_laea.tif',
             './carbon_stock_data/output_layers/soc_0_1m_t_laea.tif',
             './carbon_flux_data/output_layers/emissions_gfw_t_yr_laea.tif',
             './carbon_flux_data/output_layers/removals_gfw_t_yr_laea.tif',
             './carbon_stock_data/output_layers/carbon_sothe_spawn_t_ha_laea.tif',
             './carbon_stock_data/output_layers/soc_0_1m_t_ha_laea.tif',
             './carbon_flux_data/output_layers/emissions_gfw_t_ha_laea.tif',
             './carbon_flux_data/output_layers/removals_gfw_t_ha_laea.tif']

carbon_names = ['carbon_t',
                'soc_t',
                'emissions_t_yr',
                'removals_t_yr',
                'carbon_t_ha',
                'soc_t_ha',
                'emissions_t_ha',
                'removals_t_ha']

stat_names = ['sum',
              'sum',
              'sum',
              'sum',
              'mean',
              'mean',
              'mean',
              'mean']

In [52]:
# extract zonal stats by ecoregion
for i in range(len(carbon_fp)):
    if i == 0:
        stats_eco = extract_stats(
            dat_fp=carbon_fp[i], dat_name=carbon_names[i], vector=eco, vect_var='ECO_NAME', stat=stat_names[i])
    else:
        stats_eco = pd.concat([stats_eco,
                               extract_stats(dat_fp=carbon_fp[i], dat_name=carbon_names[i], vector=eco, vect_var='ECO_NAME', stat=stat_names[i])],
                              axis=1)

In [53]:
# extract zonal stats by biome
for i in range(len(carbon_fp)):
    if i == 0:
        stats_biome = extract_stats(
            dat_fp=carbon_fp[i], dat_name=carbon_names[i], vector=eco, vect_var='BIOME_NAME', stat=stat_names[i])
    else:
        stats_biome = pd.concat([stats_biome,
                                 extract_stats(dat_fp=carbon_fp[i], dat_name=carbon_names[i], vector=eco, vect_var='BIOME_NAME', stat=stat_names[i])],
                                axis=1)

In [54]:
# total carbon stored (+sanity check should be equal across tables)
print(stats_eco['carbon_t'].sum())
print(stats_biome['carbon_t'].sum())
print(stats_eco['soc_t'].sum())
print(stats_biome['soc_t'].sum())

5550111329.265042
5550111329.265044
28361623946.672928
28361623946.672928


In [55]:
# add total area to eco
eco['area'] = eco.geometry.area

# create new row to add to stats_eco
new_row = pd.DataFrame(
    {'ECO_NAME': eco['ECO_NAME'], 'area_km2': eco['area'] / 1e+6})
new_row.set_index('ECO_NAME', inplace=True)

# concat to stats
stats_eco = pd.concat([stats_eco, new_row], axis=1)
stats_eco

,carbon_t,soc_t,emissions_t_yr,removals_t_yr,carbon_t_ha,soc_t_ha,emissions_t_ha,removals_t_ha,area_km2
ECO_NAME,,,,,,,,,
Alberta-British Columbia foothills forests,2.356169e+08,6.285775e+08,3.220553e+06,2.661132e+06,51.875723,138.438498,85.490704,14.127711,45822.772034
Blue Mountains forests,1.413623e+08,3.259319e+08,7.678736e+05,8.469301e+05,51.002047,117.833526,101.260543,15.360629,27834.980830
British Columbia coastal conifer forests,4.251368e+07,9.087393e+07,5.129060e+04,4.845318e+05,95.729952,205.112782,100.342637,28.506463,4602.328781
Brooks-British Range tundra,7.826235e+06,1.353119e+08,2.644100e+03,1.670310e+04,17.801833,307.750048,15.862153,5.442696,4441.500901
Canadian Aspen forests and parklands,2.829307e+06,3.232623e+07,2.298000e+02,3.033440e+04,21.429108,244.942044,84.560000,20.324964,1327.364826
Central British Columbia Mountain forests,6.074951e+08,1.697009e+09,3.203280e+06,9.401294e+06,62.469650,174.624412,92.102152,27.132965,100935.318708
Fraser Plateau and Basin conifer forests,4.557070e+04,1.297867e+05,6.697000e+02,8.264000e+02,59.765574,170.211475,88.039285,24.739837,8.007105
Interior Alaska-Yukon lowland taiga,1.092599e+06,5.882381e+07,1.651130e+04,1.678170e+04,17.724394,961.860991,164.232433,17.038969,709.394041
Interior Yukon-Alaska alpine tundra,4.798406e+06,2.643832e+07,1.311100e+03,7.181260e+04,45.780841,253.180004,26.659668,17.825111,1108.112140


In [56]:
# add area to biomes
new_row = pd.DataFrame({'BIOME_NAME': eco.groupby('BIOME_NAME')['area'].sum().index.values,
                        'area_km2': eco.groupby('BIOME_NAME')['area'].sum().values / 1e+6})
new_row.set_index('BIOME_NAME', inplace=True)

# concat to stats
stats_biome = pd.concat([stats_biome, new_row], axis=1)
stats_biome

,carbon_t,soc_t,emissions_t_yr,removals_t_yr,carbon_t_ha,soc_t_ha,emissions_t_ha,removals_t_ha,area_km2
BIOME_NAME,,,,,,,,,
Temperate Conifer Forests,3.455335e+09,1.132557e+10,2.173096e+07,3.911818e+07,58.816887,193.099275,83.779040,21.312636,604242.903229
Tundra,4.738451e+08,7.451530e+09,4.380729e+06,4.899342e+06,16.939621,266.504353,102.187378,12.203409,283806.805417
"Temperate Grasslands, Savannas & Shrublands",1.061912e+08,8.694378e+08,2.376520e+05,5.605603e+05,14.147048,116.215565,104.090895,16.423994,75578.623233
Boreal Forests/Taiga,1.494365e+09,8.435799e+09,7.224619e+06,1.797060e+07,40.600871,229.473025,64.147422,14.638687,377124.755549
Deserts & Xeric Shrublands,2.037529e+07,2.792852e+08,1.791720e+04,2.862120e+04,7.046919,96.955720,87.682048,12.702642,29129.181768


In [58]:
# export to csv
stats_eco.to_excel(
    './outputs/y2y_carbon_ecoregions.xlsx', index=True)

stats_biome.to_excel(
    './outputs/y2y_carbon_biomes.xlsx', index=True)

In [20]:
# add ecoregion stats to vector data
ecoregions = eco.merge(stats_eco, on='ECO_ID')
ecoregions

,ECO_ID,ECO_NAME,BIOME_ID,BIOME_NAME,geometry,band,total_carbon,total_soc,avg_carbon_density,avg_soc_density
0,13.0,Alberta-British Columbia foothills forests,5.0,Temperate Conifer Forests,"MULTIPOLYGON (((231107.227 305009.454, 226734....",0,0.235617,0.628577,51.875723,138.438498
1,74.0,Blue Mountains forests,5.0,Temperate Conifer Forests,"POLYGON ((614252.395 -1139094.041, 612099.656 ...",0,0.141362,0.325932,51.002047,117.833526
2,83.0,British Columbia coastal conifer forests,5.0,Temperate Conifer Forests,"POLYGON ((-166421.245 37682.88, -177859.583 32...",0,0.042514,0.090874,95.729952,205.112782
3,84.0,Brooks-British Range tundra,11.0,Tundra,"MULTIPOLYGON (((-514179.871 1392834.151, -5147...",0,0.007826,0.135312,17.801833,307.750048
4,95.0,Canadian Aspen forests and parklands,8.0,"Temperate Grasslands, Savannas & Shrublands","MULTIPOLYGON (((725180.479 -339910.34, 723716....",0,0.002829,0.032326,21.429108,244.942044
5,133.0,Central British Columbia Mountain forests,5.0,Temperate Conifer Forests,"POLYGON ((192654.361 -120678.524, 194329.256 -...",0,0.607495,1.697009,62.469650,174.624412
6,272.0,Fraser Plateau and Basin conifer forests,5.0,Temperate Conifer Forests,"POLYGON ((298504.267 -341027.666, 298711.41 -3...",0,0.000046,0.000130,59.765574,170.211475
7,337.0,Interior Alaska-Yukon lowland taiga,6.0,Boreal Forests/Taiga,"MULTIPOLYGON (((-627905.705 1421020.585, -6225...",0,0.001093,0.058824,17.724394,961.860991
8,339.0,Interior Yukon-Alaska alpine tundra,11.0,Tundra,"MULTIPOLYGON (((-713662.418 1111909.509, -7138...",0,0.004798,0.026438,45.780841,253.180004
9,438.0,Mid-Canada Boreal Plains forests,6.0,Boreal Forests/Taiga,"MULTIPOLYGON (((253378.224 142341.081, 266856....",0,0.048127,0.107079,49.877737,111.053668


In [21]:
# clean up vector data
ecoregions.drop('band', axis=1, inplace=True)
ecoregions.rename(columns={'total_carbon': 'c_pg',
                           'total_soc': 'soc_pg',
                           'avg_carbon_density': 'c_t_ha',
                           'avg_soc_density': 'soc_t_ha'},
                  inplace=True)
ecoregions

,ECO_ID,ECO_NAME,BIOME_ID,BIOME_NAME,geometry,c_pg,soc_pg,c_t_ha,soc_t_ha
0,13.0,Alberta-British Columbia foothills forests,5.0,Temperate Conifer Forests,"MULTIPOLYGON (((231107.227 305009.454, 226734....",0.235617,0.628577,51.875723,138.438498
1,74.0,Blue Mountains forests,5.0,Temperate Conifer Forests,"POLYGON ((614252.395 -1139094.041, 612099.656 ...",0.141362,0.325932,51.002047,117.833526
2,83.0,British Columbia coastal conifer forests,5.0,Temperate Conifer Forests,"POLYGON ((-166421.245 37682.88, -177859.583 32...",0.042514,0.090874,95.729952,205.112782
3,84.0,Brooks-British Range tundra,11.0,Tundra,"MULTIPOLYGON (((-514179.871 1392834.151, -5147...",0.007826,0.135312,17.801833,307.750048
4,95.0,Canadian Aspen forests and parklands,8.0,"Temperate Grasslands, Savannas & Shrublands","MULTIPOLYGON (((725180.479 -339910.34, 723716....",0.002829,0.032326,21.429108,244.942044
5,133.0,Central British Columbia Mountain forests,5.0,Temperate Conifer Forests,"POLYGON ((192654.361 -120678.524, 194329.256 -...",0.607495,1.697009,62.469650,174.624412
6,272.0,Fraser Plateau and Basin conifer forests,5.0,Temperate Conifer Forests,"POLYGON ((298504.267 -341027.666, 298711.41 -3...",0.000046,0.000130,59.765574,170.211475
7,337.0,Interior Alaska-Yukon lowland taiga,6.0,Boreal Forests/Taiga,"MULTIPOLYGON (((-627905.705 1421020.585, -6225...",0.001093,0.058824,17.724394,961.860991
8,339.0,Interior Yukon-Alaska alpine tundra,11.0,Tundra,"MULTIPOLYGON (((-713662.418 1111909.509, -7138...",0.004798,0.026438,45.780841,253.180004
9,438.0,Mid-Canada Boreal Plains forests,6.0,Boreal Forests/Taiga,"MULTIPOLYGON (((253378.224 142341.081, 266856....",0.048127,0.107079,49.877737,111.053668


In [22]:
# create vector data for biomes
biomes = eco.dissolve(by=['BIOME_ID', 'BIOME_NAME'])

# reset index columns
biomes.reset_index(inplace=True)

# drop ECO_ID and ECO_NAME columns
biomes.drop(columns=['ECO_ID', 'ECO_NAME'], inplace=True)
biomes

,BIOME_ID,BIOME_NAME,geometry
0,5.0,Temperate Conifer Forests,"MULTIPOLYGON (((860037.383 -1112586.188, 86041..."
1,6.0,Boreal Forests/Taiga,"MULTIPOLYGON (((-384568.64 331052.75, -392936...."
2,8.0,"Temperate Grasslands, Savannas & Shrublands","MULTIPOLYGON (((531149.592 -972252.527, 531276..."
3,11.0,Tundra,"MULTIPOLYGON (((-274805.51 149096.379, -274330..."
4,13.0,Deserts & Xeric Shrublands,"MULTIPOLYGON (((1098249.445 -1270028.854, 1096..."


In [23]:
# add biome stats to vector data
biomes = biomes.merge(stats_biome, on='BIOME_ID')
biomes

,BIOME_ID,BIOME_NAME,geometry,band,total_carbon,total_soc,avg_carbon_density,avg_soc_density
0,5.0,Temperate Conifer Forests,"MULTIPOLYGON (((860037.383 -1112586.188, 86041...",0,3.455335,11.325572,58.816887,193.099275
1,6.0,Boreal Forests/Taiga,"MULTIPOLYGON (((-384568.64 331052.75, -392936....",0,1.494365,8.435799,40.600871,229.473025
2,8.0,"Temperate Grasslands, Savannas & Shrublands","MULTIPOLYGON (((531149.592 -972252.527, 531276...",0,0.106191,0.869438,14.147048,116.215565
3,11.0,Tundra,"MULTIPOLYGON (((-274805.51 149096.379, -274330...",0,0.473845,7.451530,16.939621,266.504353
4,13.0,Deserts & Xeric Shrublands,"MULTIPOLYGON (((1098249.445 -1270028.854, 1096...",0,0.020375,0.279285,7.046919,96.955720


In [24]:
# clean up vector data
biomes.drop('band', axis=1, inplace=True)
biomes.rename(columns={'total_carbon': 'c_pg',
                       'total_soc': 'soc_pg',
                       'avg_carbon_density': 'c_t_ha',
                       'avg_soc_density': 'soc_t_ha'},
              inplace=True)
biomes

,BIOME_ID,BIOME_NAME,geometry,c_pg,soc_pg,c_t_ha,soc_t_ha
0,5.0,Temperate Conifer Forests,"MULTIPOLYGON (((860037.383 -1112586.188, 86041...",3.455335,11.325572,58.816887,193.099275
1,6.0,Boreal Forests/Taiga,"MULTIPOLYGON (((-384568.64 331052.75, -392936....",1.494365,8.435799,40.600871,229.473025
2,8.0,"Temperate Grasslands, Savannas & Shrublands","MULTIPOLYGON (((531149.592 -972252.527, 531276...",0.106191,0.869438,14.147048,116.215565
3,11.0,Tundra,"MULTIPOLYGON (((-274805.51 149096.379, -274330...",0.473845,7.451530,16.939621,266.504353
4,13.0,Deserts & Xeric Shrublands,"MULTIPOLYGON (((1098249.445 -1270028.854, 1096...",0.020375,0.279285,7.046919,96.955720


In [25]:
# write to file
ecoregions.to_file(
    './carbon_stock_data/output_layers/ecoregions_biomes/ecoregions_carbon_stock.shp')
biomes.to_file(
    './carbon_stock_data/output_layers/ecoregions_biomes/biomes_carbon_stock.shp')

In [37]:
# add area to biomes table in ha
# 1 m2 = 0.0001 ha
biomes['area_ha'] = biomes.area / 10000
biomes['area_pc'] = biomes['area_ha'] / biomes['area_ha'].sum()
biomes

,BIOME_ID,BIOME_NAME,geometry,c_pg,soc_pg,c_t_ha,soc_t_ha,area_ha,area_pc
0,5.0,Temperate Conifer Forests,"MULTIPOLYGON (((860037.383 -1112586.188, 86041...",3.455335,11.325572,58.816887,193.099275,6.042429e+07,0.441091
1,6.0,Boreal Forests/Taiga,"MULTIPOLYGON (((-384568.64 331052.75, -392936....",1.494365,8.435799,40.600871,229.473025,3.771248e+07,0.275297
2,8.0,"Temperate Grasslands, Savannas & Shrublands","MULTIPOLYGON (((531149.592 -972252.527, 531276...",0.106191,0.869438,14.147048,116.215565,7.557862e+06,0.055172
3,11.0,Tundra,"MULTIPOLYGON (((-274805.51 149096.379, -274330...",0.473845,7.451530,16.939621,266.504353,2.838068e+07,0.207176
4,13.0,Deserts & Xeric Shrublands,"MULTIPOLYGON (((1098249.445 -1270028.854, 1096...",0.020375,0.279285,7.046919,96.955720,2.912918e+06,0.021264


In [90]:
# drop geometry
biomes_df = biomes.drop('geometry', axis=1)

# add totals row
biomes_df = pd.concat(
    [biomes_df, biomes_df.sum().to_frame().transpose()], ignore_index=True)
biomes_df

,BIOME_ID,BIOME_NAME,c_pg,soc_pg,c_t_ha,soc_t_ha,area_ha,area_pc
0,5.0,Temperate Conifer Forests,3.455335,11.325572,58.816887,193.099275,60424290.322885,0.441091
1,6.0,Boreal Forests/Taiga,1.494365,8.435799,40.600871,229.473025,37712475.554948,0.275297
2,8.0,"Temperate Grasslands, Savannas & Shrublands",0.106191,0.869438,14.147048,116.215565,7557862.323312,0.055172
3,11.0,Tundra,0.473845,7.45153,16.939621,266.504353,28380680.541698,0.207176
4,13.0,Deserts & Xeric Shrublands,0.020375,0.279285,7.046919,96.95572,2912918.176846,0.021264
5,43.0,Temperate Conifer ForestsBoreal Forests/TaigaT...,5.550111,28.361624,137.551346,902.247939,136988226.919689,1.0


In [105]:
# clean up Y2Y row names
biomes_df.loc[len(biomes_df) - 1, 'BIOME_ID'] = 0
biomes_df.loc[len(biomes_df) - 1, 'BIOME_NAME'] = 'Y2Y'

# change Y2Y rows to average for density
biomes_df.loc[len(biomes_df) - 1,
              'c_t_ha'] = biomes_df.loc[:len(biomes_df) - 2, 'c_t_ha'].mean()
biomes_df.loc[len(biomes_df) - 1,
              'soc_t_ha'] = biomes_df.loc[:len(biomes_df) - 2, 'soc_t_ha'].mean()

# get all columns except biome_name
cols = [col for col in biomes_df.columns if col != 'BIOME_NAME']

# convert to numeric
biomes_df[cols] = biomes_df[cols].apply(pd.to_numeric)

# round columns
decimals = pd.Series([1, 1, 1, 1, 1, 2], index=[
                     'c_pg', 'soc_pg', 'c_t_ha', 'soc_t_ha', 'area_ha', 'area_pc'])
biomes_df = biomes_df.round(decimals)
biomes_df

,BIOME_ID,BIOME_NAME,c_pg,soc_pg,c_t_ha,soc_t_ha,area_ha,area_pc
0,5.0,Temperate Conifer Forests,3.5,11.3,58.8,193.1,60424290.3,0.44
1,6.0,Boreal Forests/Taiga,1.5,8.4,40.6,229.5,37712475.6,0.28
2,8.0,"Temperate Grasslands, Savannas & Shrublands",0.1,0.9,14.1,116.2,7557862.3,0.06
3,11.0,Tundra,0.5,7.5,16.9,266.5,28380680.5,0.21
4,13.0,Deserts & Xeric Shrublands,0.0,0.3,7.0,97.0,2912918.2,0.02
5,0.0,Y2Y,5.6,28.4,27.5,180.5,136988226.9,1.00


In [107]:
# export to csv
biomes_df.to_csv(
    './carbon_stock_data/output_layers/biomes_carbon_stocks.csv', index=False)